# Carbon Investment Potential
Data preparation for the carbon investment potential widget (pie/donut chart).  
Data model is as follows:  

location_id: staging API id  
category: carbon_5,   
area: int
unit: ha  
note: sentence for 5$ potential

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import os


In [52]:
old_carbon = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/mangrove_data_v2_join.csv')
old_carbon.head()

,id,date,gain_m2,loss_m2,length_m,area_m2,hmax_m,agb_mgha_1,hba_m,location_id,...,hba_hist_m,hmax_hist_m,total_carbon,agb_tco2e,bgb_tco2e,soc_tco2e,toc_tco2e,toc_hist_tco2eha,iso,location_type
0,39237,2000-01-01,NaN,NaN,NaN,NaN,11.006700,35.453350,NaN,1105,...,NaN,"[[0,19349.180392156864],[1,19304.67450980392],...",NaN,NaN,NaN,NaN,NaN,NaN,USA,wdpa
1,39238,2007-01-01,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000,0.000000,NaN,1106,...,NaN,"[[0,0],[1,0],[2,0],[3,0],[4,0],[5,0],[6,0],[7,...",NaN,NaN,NaN,NaN,NaN,NaN,VGB,wdpa
2,39239,2008-01-01,0.000000e+00,0.000000e+00,0.00,0.000000e+00,0.000000,0.000000,NaN,1106,...,NaN,"[[0,0],[1,0],[2,0],[3,0],[4,0],[5,0],[6,0],[7,...",NaN,NaN,NaN,NaN,NaN,NaN,VGB,wdpa
3,39365,2000-01-01,NaN,NaN,NaN,NaN,9.389063,54.013480,NaN,1121,...,NaN,"[[0,9122.572549019611],[1,12830.776470588235],...",NaN,NaN,NaN,NaN,NaN,NaN,GNB,wdpa
4,37651,2016-01-01,1.060384e+07,1.328997e+07,691109.54,1.752985e+09,18.773118,178.978127,NaN,907,...,NaN,"[[0,11656],[1,20860.066666666666],[2,0],[3,677...",NaN,5.181886e+07,4.198865e+07,2.858326e+08,3.376514e+08,"{""0--700"":74437,""1400--2100"":1212295,""2100--28...",GAB,country


In [53]:
old_carbon.columns

Index(['id', 'date', 'gain_m2', 'loss_m2', 'length_m', 'area_m2', 'hmax_m',
       'agb_mgha_1', 'hba_m', 'location_id', 'created_at', 'updated_at',
       'con_hotspot_summary_km2', 'net_change_m2', 'agb_hist_mgha_1',
       'hba_hist_m', 'hmax_hist_m', 'total_carbon', 'agb_tco2e', 'bgb_tco2e',
       'soc_tco2e', 'toc_tco2e', 'toc_hist_tco2eha', 'iso', 'location_type'],
      dtype='object')

In [56]:
old_carbon = old_carbon[old_carbon.toc_hist_tco2eha.notnull()][['location_id', 'toc_hist_tco2eha']]
old_carbon

,location_id,toc_hist_tco2eha
4,907,"{""0--700"":74437,""1400--2100"":1212295,""2100--28..."
15,893,"{""0--700"":19064,""1400--2100"":247482,""2100--280..."
23,894,"{""0--700"":29051,""1400--2100"":61577,""2100--2800..."
31,895,"{""0--700"":8761,""1400--2100"":21961,""2100--2800""..."
39,896,"{""0--700"":96755,""1400--2100"":1038230,""2100--28..."
...,...,...
2065,1150,"{""0--700"":0,""1400--2100"":0,""2100--2800"":0,""280..."
2073,1151,"{""0--700"":99,""1400--2100"":600,""2100--2800"":151..."
2081,1152,"{""0--700"":0,""1400--2100"":0,""2100--2800"":0,""280..."
2089,1153,"{""0--700"":0,""1400--2100"":0,""2100--2800"":0,""280..."


In [26]:
carb_columns = old_carbon['toc_hist_tco2eha'].map(eval).apply(pd.Series)
carb_columns

,0--700,1400--2100,2100--2800,2800--3500,700--1400
4,74437,1212295,362360,137176,109765
15,19064,247482,132289,2908,191427
23,29051,61577,1,0,608041
31,8761,21961,613,0,45658
39,96755,1038230,479301,222945,347928
...,...,...,...,...,...
2065,0,0,0,0,0
2073,99,600,1513,5,46
2081,0,0,0,0,0
2089,0,0,0,0,0


In [30]:
old_carbon_full = pd.concat([old_carbon['location_id'], carb_columns], axis=1).reset_index().drop(columns = 'index')
old_carbon_full.head()

,location_id,0--700,1400--2100,2100--2800,2800--3500,700--1400
0,907,74437,1212295,362360,137176,109765
1,893,19064,247482,132289,2908,191427
2,894,29051,61577,1,0,608041
3,895,8761,21961,613,0,45658
4,896,96755,1038230,479301,222945,347928


In [32]:
carbon_long = pd.melt(old_carbon_full, id_vars='location_id')
carbon_long['variable'] = carbon_long['variable'].str.replace('--', '-')
carbon_long.rename(columns={'variable':'indicator'}, inplace=True)
carbon_long

,location_id,indicator,value
0,907,0-700,74437
1,893,0-700,19064
2,894,0-700,29051
3,895,0-700,8761
4,896,0-700,96755
...,...,...,...
1310,1150,700-1400,0
1311,1151,700-1400,46
1312,1152,700-1400,0
1313,1153,700-1400,0


## Add Locations (old --> new)  
Add ISO code using old location  
Add new location_id using ISO

In [40]:
OLDdataLocation = requests.get('https://mangrove-atlas-api.herokuapp.com/api/v1/locations').json()['data']
#dataLocation = requests.get('http://192.168.50.115:3000/api/v2/locations').json()['data']
old_locations = pd.DataFrame(OLDdataLocation)
old_locations = old_locations[old_locations['location_type'] == 'country']
old_locations = old_locations[['id', 'iso']]
old_locations

,id,iso
0,1155,WORLDWIDE
1,1012,ARE
2,1114,BRA
3,992,AGO
4,964,ATG
...,...,...
258,1140,CHN
259,1067,ECU
260,1061,MEX
261,1062,MEX


In [44]:
df_join = pd.merge(carbon_long, old_locations, left_on='location_id', right_on='id', how = 'inner')
df_join.drop(columns=['id'], inplace=True)
df_join.rename(columns={'location_id':'old_location_id'}, inplace=True)
df_join

,old_location_id,indicator,value,iso
0,907,0-700,74437,GAB
1,907,1400-2100,1212295,GAB
2,907,2100-2800,362360,GAB
3,907,2800-3500,137176,GAB
4,907,700-1400,109765,GAB
...,...,...,...,...
1310,1154,0-700,27623,GUF
1311,1154,1400-2100,21786,GUF
1312,1154,2100-2800,48,GUF
1313,1154,2800-3500,0,GUF


Add new locations

In [45]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [46]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [59]:
df_final = pd.merge(df_join, locations[['iso', 'location_idn']], on='iso', how='left')
df_final = pd.merge(df_final, api_locs, on='location_idn', how='left')
df_final.head()

,old_location_id,indicator,value,iso,location_idn,id
0,907,0-700,74437,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0
1,907,1400-2100,1212295,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0
2,907,2100-2800,362360,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0
3,907,2800-3500,137176,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0
4,907,700-1400,109765,GAB,94ad2556-67f6-5c6e-8145-dad0ef636398,3400.0


In [61]:
df_final[df_final.id.isnull()]

,old_location_id,indicator,value,iso,location_idn,id
350,962,0-700,4209,-,NaN,NaN
351,962,1400-2100,117981,-,NaN,NaN
352,962,2100-2800,3978,-,NaN,NaN
353,962,2800-3500,0,-,NaN,NaN
354,962,700-1400,8757,-,NaN,NaN
1205,1155,0-700,58599882,WORLDWIDE,NaN,NaN
1206,1155,1400-2100,36665682,WORLDWIDE,NaN,NaN
1207,1155,2100-2800,13668129,WORLDWIDE,NaN,NaN
1208,1155,2800-3500,58026013,WORLDWIDE,NaN,NaN
1209,1155,700-1400,5996751,WORLDWIDE,NaN,NaN


In [62]:
df_final.drop(columns=['iso', 'location_idn', 'old_location_id'], inplace=True)
df_final.rename(columns={'id':'location_id'}, inplace=True)
df_final = df_final[df_final.location_id.notnull()]
#df_final['unit'] = 'ha'
df_final.head()

,indicator,value,location_id
0,0-700,74437,3400.0
1,1400-2100,1212295,3400.0
2,2100-2800,362360,3400.0
3,2800-3500,137176,3400.0
4,700-1400,109765,3400.0


In [64]:
df_final.to_csv('../../../../data/OLD_blue_carbon_UPDATE.csv', index=False)